# Extend AWS HealthScribe Response with Comprehend Medical

> *This notebook is compatible with SageMaker kernels `Data Science 3.0` or `conda_python3` on an `ml.t3.medium` instance*

## Introduction

This notebook shows how to integrate the output generated by AWS HealthScribe with Amazon Comprehend Medical.

In [1]:
# retrieve plan from previous notebook
%store -r plan
print(plan)


Lower back pain
  - Start physical therapy for a minimum of 4 weeks, ideally 6 weeks, to help relieve the lower back pain without waiting for the X-ray results. 

  - Take an X-ray of the lower back to check for any underlying issues causing the pain.

  - Continue taking over-the-counter pain medications as needed in the meantime.

  - Use a chair with lumbar support or position a pillow for lumbar support while sitting for long periods.

Sleep
  - Improve sleep habits by separating from the toddler at night to allow uninterrupted sleep and a more comfortable sleeping position.

Exercise
  - Start light exercises like walking or yoga to improve posture and handle the pain.

Follow up
  - Reassess the situation after physical therapy - if pain does not improve, order a CT scan of the lower back to further investigate the likely cause of sciatica (pinched nerve in the lower back radiating pain to legs).



## 2. Combining HealthScribe with Comprehend Medical

Call ICD-10 and RxNorm ontologies in Comprehend Medical with the raw content of the transcript.

### 2.1. ICD-10-CM linking
When medical conditions are detected, InferICD10CM returns the matching ICD-10-CM codes and descriptions. The detected conditions are listed in descending order of confidence. The scores indicate the confidence in the accuracy of the entities matched to the concepts found in the text. Related information such as family history, signs, symptoms, and negation are recognized as traits. Additional information such as anatomical designations and acuity are listed as attributes.

**Note:** depending on the audio file used, you might get an empty output as a result of this method call in case there are no matches with the ontology.

In [2]:
import boto3
import json

# connect to the Amazon Comprehend Medical service in the us-east-1 region
cm_client = boto3.client(service_name='comprehendmedical', region_name='us-east-1')

result = cm_client.infer_icd10_cm(Text= plan)
entities = result['Entities'];
print(json.dumps(entities, indent=2))

[
  {
    "Id": 3,
    "Text": "Lower back pain",
    "Category": "MEDICAL_CONDITION",
    "Type": "DX_NAME",
    "Score": 0.9201511740684509,
    "BeginOffset": 1,
    "EndOffset": 16,
    "Attributes": [
      {
        "Type": "DIRECTION",
        "Score": 0.7950295805931091,
        "RelationshipScore": 0.7814922332763672,
        "Id": 1,
        "BeginOffset": 1,
        "EndOffset": 6,
        "Text": "Lower",
        "Traits": []
      },
      {
        "Type": "SYSTEM_ORGAN_SITE",
        "Score": 0.90631103515625,
        "RelationshipScore": 0.9234480857849121,
        "Id": 2,
        "BeginOffset": 1,
        "EndOffset": 11,
        "Text": "Lower back",
        "Traits": []
      }
    ],
    "Traits": [
      {
        "Name": "SYMPTOM",
        "Score": 0.9596198797225952
      }
    ],
    "ICD10CMConcepts": [
      {
        "Description": "Low back pain, unspecified",
        "Code": "M54.50",
        "Score": 0.8267222046852112
      },
      {
        "Descriptio

### 2.2. RxNorm linking
Amazon Comprehend Medical lists the top potentially matching RxCUIs for each medication that it detects in descending order by confidence score. Use the RxCUI codes for downstream analysis that is not possible with unstructured text. Related information such as strength, frequency, dose, dose form, and route of administration are listed as attributes in JSON format.

**Note:** depending on the audio file used, you might get an empty output as a result of this method call in case there are no matches with the ontology.

In [3]:
result = cm_client.infer_rx_norm(Text= plan)
entities = result['Entities'];
print(json.dumps(entities, indent=2))

[]
